In [1]:
import json
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

In [2]:
key = open('key_백관구.txt').read()
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'
queryParams = f"?{quote_plus('ServiceKey')}={key}&" +\
              urlencode({quote_plus('pageNo') : '1',\
						 quote_plus('numOfRows') : '300',\
						 quote_plus('dataType') : 'JSON',\
						 quote_plus('base_date') : '20210224',\
						 quote_plus('base_time') : '1700',\
						 quote_plus('nx') : '62',\
						 quote_plus('ny') : '121' })
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read().decode('utf-8')
print(response_body)

e":"20210224","baseTime":"1700","category":"SKY","fcstDate":"20210225","fcstTime":"0600","fcstValue":"4","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"T3H","fcstDate":"20210225","fcstTime":"0600","fcstValue":"-1","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"TMN","fcstDate":"20210225","fcstTime":"0600","fcstValue":"-1.0","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"UUU","fcstDate":"20210225","fcstTime":"0600","fcstValue":"-0.9","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"VEC","fcstDate":"20210225","fcstTime":"0600","fcstValue":"90","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"VVV","fcstDate":"20210225","fcstTime":"0600","fcstValue":"0","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"WSD","fcstDate":"20210225","fcstTime":"0600","fcstValue":"1","nx":62,"ny":121},{"baseDate":"20210224","baseTime":"1700","category":"POP","fcstDate":"2021022

In [10]:
json.loads(response_body)['response'].keys()

dict_keys(['header', 'body'])

In [3]:
response_df = pd.DataFrame(json.loads(response_body)['response']['body']['items']['item'])
response_df['DT_base'] = response_df.apply(lambda x: x['baseDate'] + x['baseTime'], axis = 1)
response_df['DT_fcst'] = response_df.apply(lambda x: x['fcstDate'] + x['fcstTime'], axis = 1)

pivot_df = response_df.pivot(index = 'DT_fcst', columns = 'category', values = 'fcstValue')
pivot_df['DT_base'] = response_df['DT_base'][0]
pivot_df['NX']      = response_df['nx'][0]
pivot_df['NY']      = response_df['ny'][0]
pivot_df.reset_index(inplace = True)
pivot_df.sort_index(axis = 1, inplace = True)
pivot_df.columns.name = None
pivot_df

,DT_base,DT_fcst,NX,NY,POP,PTY,R06,REH,S06,SKY,T3H,TMN,TMX,UUU,VEC,VVV,WSD
0,202102241700,202102242100,62,121,0,0,NaN,65,NaN,1,1,NaN,NaN,0,360,-1,1
1,202102241700,202102250000,62,121,0,0,0,60,0,1,0,NaN,NaN,-1.4,94,0.1,1.5
2,202102241700,202102250300,62,121,10,0,NaN,70,NaN,1,0,NaN,NaN,-1.3,90,0,1.4
3,202102241700,202102250600,62,121,30,0,0,85,0,4,-1,-1.0,NaN,-0.9,90,0,1
4,202102241700,202102250900,62,121,20,0,NaN,75,NaN,3,2,NaN,NaN,-1.3,117,0.7,1.6
5,202102241700,202102251200,62,121,20,0,0,60,0,3,6,NaN,NaN,-1.5,114,0.7,1.7
6,202102241700,202102251500,62,121,20,0,NaN,50,NaN,3,8,NaN,9.0,-0.5,90,0,0.6
7,202102241700,202102251800,62,121,20,0,0,65,0,3,7,NaN,NaN,0.6,304,-0.4,0.7
8,202102241700,202102252100,62,121,20,0,NaN,75,NaN,3,4,NaN,NaN,-0.3,24,-0.9,1
9,202102241700,202102260000,62,121,20,0,0,75,0,3,2,NaN,NaN,-0.8,66,-0.4,1


In [5]:
from kma_api_pkg import kma_api

kma_api.load('20210224', '1700', '62', '121').run()

,DT_base,DT_fcst,NX,NY,POP,PTY,R06,REH,S06,SKY,T3H,TMN,TMX,UUU,VEC,VVV,WSD
0,202102241700,202102242100,62,121,0,0,NaN,65,NaN,1,1,NaN,NaN,0,360,-1,1
1,202102241700,202102250000,62,121,0,0,0,60,0,1,0,NaN,NaN,-1.4,94,0.1,1.5
2,202102241700,202102250300,62,121,10,0,NaN,70,NaN,1,0,NaN,NaN,-1.3,90,0,1.4
3,202102241700,202102250600,62,121,30,0,0,85,0,4,-1,-1.0,NaN,-0.9,90,0,1
4,202102241700,202102250900,62,121,20,0,NaN,75,NaN,3,2,NaN,NaN,-1.3,117,0.7,1.6
5,202102241700,202102251200,62,121,20,0,0,60,0,3,6,NaN,NaN,-1.5,114,0.7,1.7
6,202102241700,202102251500,62,121,20,0,NaN,50,NaN,3,8,NaN,9.0,-0.5,90,0,0.6
7,202102241700,202102251800,62,121,20,0,0,65,0,3,7,NaN,NaN,0.6,304,-0.4,0.7
8,202102241700,202102252100,62,121,20,0,NaN,75,NaN,3,4,NaN,NaN,-0.3,24,-0.9,1
9,202102241700,202102260000,62,121,20,0,0,75,0,3,2,NaN,NaN,-0.8,66,-0.4,1
